In [1]:
import json
import fileinput
import requests
from requests.exceptions import Timeout
import re
import os
import time

# helper function to convert our text files to json
def text2json(text):
    fin = fileinput.input(text, inplace=1)
    for line in fin:
        if fin.isfirstline():
            print ("[" + line)
        else:
            print ("," + line)
    fin.close()
    
    f = open(text,"a")
    f.write("]")
    f.close()
    
# load json from textfile
def loaddata(text): 
    # if json.load fails, format
    try:
        f = open(text, 'r')
        data = json.load(f)
        f.close()
        return data 
    except:
        print("Formatting " + text + "...")
        text2json(text)
        return loaddata(text)

# load title from url
def gettitle(url):
    try:
        r = requests.get(url, timeout=1.5)
        if r:
            return re.findall("(?<=<title>)\s*(.*)\s*(?=<\/title>)", r.text)
        else:
            return ''
    except Timeout: 
        print("Request timed out")
        return ''
    except Exception as e: 
        print("Exception: ", str(e))
        return ''
        
    # return text between <title> tags in html 

# add url title to ['title'] attribute, textfile to new file
def addtitle(textfile, outdir, counter):
    data = loaddata(textfile)
    try:
        print("Starting to convert file " + textfile)
        for tweet in data:
            titles = []
            for url in tweet['tweet'][0]['entities']['urls']:
                title = gettitle(url['expanded_url'])
                titles.append(title)
            tweet['titles'] = titles
        outfile = outdir + "t-" + str(counter) + ".json"
        with open(outfile, 'w') as file:
             file.write(json.dumps(data)) # use `json.loads` to do the reverse
        print("Finished converting file " + textfile)
    except Exception as e:
        print("Exception thrown: " + str(e))
    

In [2]:
import os

# set up dir
# os.chdir("tweets")
directory = os.getcwd()
files = os.listdir(directory)
counter = 28
odir = "../convertedtweets/"

for f in files:
    filename, file_extension = os.path.splitext(f)
    if(file_extension == '.txt'):
        addtitle(f, odir, counter)
        counter += 1

['t2-9.txt', 't2-57.txt', 't2-43.txt', 't2-42.txt', 't2-56.txt', 't2-8.txt', 't2-52.txt', 't2-46.txt', 't2-47.txt', 't2-53.txt', 't2-45.txt', 't2-51.txt', 't2-50.txt', 't2-44.txt', 't2-3.txt', 't2-49.txt', 't2-61.txt', 't2-60.txt', 't2-48.txt', 't2-2.txt', 't2-62.txt', 't2-63.txt', 't2-1.txt', 't2-5.txt', 't2-67.txt', 't2-66.txt', 't2-4.txt', 't2-6.txt', 't2-64.txt', 't2-70.txt', 't2-58.txt', 't2-59.txt', 't2-71.txt', 't2-65.txt', 't2-7.txt', '.ipynb_checkpoints', 't-3.txt', 't1-2.txt', 't2-16.txt', 't2-17.txt', 't1-3.txt', 't-2.txt', 't1-1.txt', 't2-29.txt', 't2-15.txt', 't2-14.txt', 't2-28.txt', 't-1.txt', 't-5.txt', 't1-4.txt', 't2-10.txt', 't2-38.txt', 't2-39.txt', 't2-11.txt', 't1-5.txt', 't-4.txt', 't-6.txt', 't1-7.txt', 't2-13.txt', 't2-12.txt', 't1-6.txt', 't-7.txt']
